In [1]:
from pathlib import Path
import sys
import os
sys.path.append(os.path.abspath("..")) 

import numpy as np
import pandas as pd
import itertools
import re

from scripts.get_paths import get_path
from scripts.agreement import compute_series_agreement
from scripts.nod_detector import NodDetector

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
paths = get_path()

In [3]:
grid = {
    "k_std_velocity":[1.0,2.0,3.0,4.0],
    "k_std_still_pitch":[1.0,2.0,3.0,4.0]
}

In [4]:
results_openface = []
param_names = list(grid.keys())
for vals in itertools.product(*(grid[k] for k in param_names)):
    params = dict(zip(param_names, vals))
   
    detector = NodDetector(**params)

    df = pd.read_csv(paths.features_2 / "extracted_features.csv", index_col=False)
    df = df[df['NT']==1]
    df = df.sort_values(['participant_id', 'clip', 'frame'], kind='mergesort')

    df['nod_openface_grid_search'] = 0

    for (participant_id, clip), df_sub in df.groupby(['participant_id', 'clip'], sort=False):

        fps = (df_sub['frame'].iloc[-1] - df_sub['frame'].iloc[0]) / (
                df_sub['timestamp'].iloc[-1] - df_sub['timestamp'].iloc[0])
        
        pitch_openface = df_sub["pitch_openface"].to_numpy()
        yaw_openface = df_sub["yaw_openface"].to_numpy()

        # If your detector can’t handle NaNs, skip or fill:
        if np.isnan(pitch_openface).any() or np.isnan(yaw_openface).any():
            pitch_openface = pd.Series(pitch_openface).interpolate(limit_direction="both").to_numpy()
            yaw_openface   = pd.Series(yaw_openface).interpolate(limit_direction="both").to_numpy()

        events_openface, _ = detector.detect_nod(
            pitch=pitch_openface,
            yaw=yaw_openface,
            fps=fps,
            radians=True
        )
        for start, end in events_openface:
            idx = df_sub.index[start:end+1]   # index positions -> rows
            df.loc[idx, 'nod_openface_grid_search'] = 1

    agreement_score = compute_series_agreement(df, 'nod', 'nod_agreed', 'nod_openface_grid_search')
    results_openface.append({**params, "agreement_score": round(agreement_score['agreement_score']*100)})
        

In [5]:
top5_openface = sorted(
    results_openface,
    key=lambda d: d["agreement_score"],
    reverse=True
)[:3]
top5_openface

[{'k_std_velocity': 1.0, 'k_std_still_pitch': 4.0, 'agreement_score': 59},
 {'k_std_velocity': 2.0, 'k_std_still_pitch': 4.0, 'agreement_score': 59},
 {'k_std_velocity': 3.0, 'k_std_still_pitch': 3.0, 'agreement_score': 59}]

In [6]:
results_pyafar = []
param_names = list(grid.keys())
for vals in itertools.product(*(grid[k] for k in param_names)):
    params = dict(zip(param_names, vals))
    
    detector = NodDetector(**params)

    df = pd.read_csv(paths.features_2 / "extracted_features.csv", index_col=False)
    df = df[df['NT']==1]
    df = df.sort_values(['participant_id', 'clip', 'frame'], kind='mergesort')

    df['nod_pyafar_grid_search'] = 0

    for (participant_id, clip), df_sub in df.groupby(['participant_id', 'clip'], sort=False):

        fps = (df_sub['frame'].iloc[-1] - df_sub['frame'].iloc[0]) / (
                df_sub['timestamp'].iloc[-1] - df_sub['timestamp'].iloc[0])

        pitch_pyafar = df_sub["pitch_pyafar"].to_numpy()
        yaw_pyafar = df_sub["yaw_pyafar"].to_numpy()

        # If your detector can’t handle NaNs, skip or fill:
        if np.isnan(pitch_pyafar).any() or np.isnan(yaw_pyafar).any():
            pitch_pyafar = pd.Series(pitch_pyafar).interpolate(limit_direction="both").to_numpy()
            yaw_pyafar   = pd.Series(yaw_pyafar).interpolate(limit_direction="both").to_numpy()

        events_pyafar, _ = detector.detect_nod(
            pitch=pitch_pyafar,
            yaw=yaw_pyafar,
            fps=fps,
            radians=False
        )
        for start, end in events_pyafar:
            idx = df_sub.index[start:end+1]
            df.loc[idx, 'nod_pyafar_grid_search'] = 1

    agreement_score = compute_series_agreement(df, 'nod', 'nod_agreed', 'nod_pyafar_grid_search')
    results_pyafar.append({**params, "agreement_score": round(agreement_score['agreement_score']*100)})
        

In [7]:
top5_pyafar = sorted(
    results_pyafar,
    key=lambda d: d["agreement_score"],
    reverse=True
)[:3]
top5_pyafar

[{'k_std_velocity': 1.0, 'k_std_still_pitch': 4.0, 'agreement_score': 62},
 {'k_std_velocity': 2.0, 'k_std_still_pitch': 4.0, 'agreement_score': 62},
 {'k_std_velocity': 3.0, 'k_std_still_pitch': 4.0, 'agreement_score': 62}]